In [29]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Author: lvhaili
# @Date  : 2018/1/3
# @contact: lvhaili@foryou56.com
# @Desc  :  数据库连接配置

import time
import threading
import pymysql
from sshtunnel import SSHTunnelForwarder, create_logger
import pandas as pd

sshIp = 'IP'
sshPort = 22
sshUser = 'root'
sshPassword = 'password'
localPort = 10000  # 该数字随意


class DbVo(object):
	def __init__(self,
				 host=None,
				 port=None,
				 user=None,
				 password=None,
				 dbName=None,
				 charset='utf-8',
				 localPort=10000
				 ):
		self.host = host
		self.port = port
		self.user = user
		self.password = password
		self.dbName = dbName
		self.charset = charset
		self.localPort = localPort

def createSSH(proxyInfo):
    return  SSHTunnelForwarder(
		ssh_address_or_host=(sshIp, sshPort),
		ssh_username=sshUser,
		ssh_password=sshPassword,
		local_bind_address=('127.0.0.1', proxyInfo.localPort),  # 绑定的端口
		remote_bind_address=(proxyInfo.host, proxyInfo.port),
		# 代理远程的端口
		logger=create_logger(loglevel='ERROR')  # sshtunnel打开日志输出
	)


def sshconnect(server):
	server.start()
	print  ('sub thread start!the thread name is:%s\r' % threading.currentThread().getName())


def proxyConnect(sshserver, proxyInfo):
	thread = threading.Thread(target=sshconnect, args=(sshserver,))
	# thread.setDaemon(True)
	thread.start()
	#   thread.join()
	print ('ssh service start...')


def connectDataBase(dbinfo):
	time.sleep(2)
	client = pymysql.connect(
		host='127.0.0.1',  # 此处必须是是127.0.0.1
		port=dbinfo.localPort,
		user=dbinfo.user,
		passwd=dbinfo.password,
		db=dbinfo.dbName,
		charset='utf8')
	return client


def getData(dbClient, tableName):
    sql = "select * from " + tableName + " limit 10"
    print (sql)
    cursor = dbClient.cursor()
    cursor.execute(sql)
    values = cursor.fetchall()
    cursor.close()
    print (values[0])
    return values
    # dbClient.close()


dataCenter = DbVo(
	host="ip",  # 此处的host没有用上，因为用到的是上面的127.0.0.1
	port=3306,
	user="user",
	password="password",
	dbName="database",
	localPort=8203 #该数字随意
)

def getClient():
	ssh = createSSH(dataCenter)
	proxyConnect(ssh, dataCenter)
	dataCenterDbClient = connectDataBase(dataCenter)
	return ssh, dataCenterDbClient

In [30]:
# 客户端调用实例
if __name__ == '__main__':
	ssh, dataCenterDbClient = getClient()
	data = getData(dataCenterDbClient, 'bankaccountjournal')
	dataCenterDbClient.close()
	ssh.close()
	print( 'begin exit ')

ssh service start...
sub thread start!the thread name is:Thread-18
select * from bankaccountjournal limit 10
(1, datetime.datetime(2016, 11, 9, 0, 0), 102976.8, 498, datetime.datetime(2016, 11, 10, 9, 32, 2), 2, -1)
begin exit 


In [31]:
pd.DataFrame(list(data))

,0,1,2,3,4,5,6
0,1,2016-11-09 00:00:00,102976.80,498.0,2016-11-10 09:32:02,2,-1
1,2,2016-11-10 00:00:00,2677902.12,NaN,NaT,0,1
2,3,2016-11-10 00:00:00,0.72,NaN,NaT,0,3
3,4,2016-11-11 00:00:01,20633768.06,NaN,NaT,0,1
4,5,2016-11-12 00:00:00,7208149.75,NaN,NaT,0,1
5,6,2016-11-14 00:00:01,8526.00,NaN,NaT,0,-1
6,7,2016-11-14 00:00:01,11925597.69,NaN,NaT,0,1
7,8,2016-11-15 00:00:01,7753066.29,NaN,NaT,0,-1
8,9,2016-11-15 00:00:01,11583493.78,NaN,NaT,0,1
9,10,2016-11-16 00:00:00,11780503.95,NaN,NaT,0,-1
